In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

storm_df = pd.read_csv("/kaggle/input/noaa-powout-prism-0-1-is-storm-lag/noaapowoutprism_01_Is_Storm_Lag (1).csv")

# Storm Prediction

In [2]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

# Define features
storm_features = [
    'DEATHS_INDIRECT', 'INJURIES_DIRECT', 'INJURIES_INDIRECT', 'DEATHS_DIRECT',
    'DAMAGE_PROPERTY', 'DAMAGE_CROPS', 'customers_out', 'duration_hours',
    'desc_word_count', 'desc_char_count',
    'has_tornado', 'has_hail', 'has_flood', 'has_wind', 'has_tree',
    'has_power', 'has_damage', 'has_outage', 'has_broken', 'has_blown',
    'tmin', 'tmax', 'tavg', 'ppt'
]

# Prepare X and y
X = storm_df[storm_features]
y = storm_df['is_storm_lagged'].astype(int)

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [3]:
# Train Random Forest model on all data with best hyperparameters obtained using optuna
rf_model = RandomForestClassifier(
    n_estimators=188,
    max_depth=50,
    min_samples_split=2,
    min_samples_leaf=1,
    max_features='log2',
    bootstrap=False,
    random_state=42,
    n_jobs=-1
)
rf_model.fit(X_scaled, y)

# Predict on all data
predictions = rf_model.predict(X_scaled)

# Create new DataFrame with predictions
storm_df_with_predictions = storm_df.copy()
storm_df_with_predictions['predicted_storm'] = predictions

In [4]:
storm_df_with_1_predictions = storm_df_with_predictions[storm_df_with_predictions['predicted_storm'] == 1].copy()

# Severity Prediction

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

severity_features = [
    # Original impact features
    'DEATHS_INDIRECT', 'INJURIES_DIRECT', 'INJURIES_INDIRECT', 'DEATHS_DIRECT',
    'DAMAGE_PROPERTY', 'DAMAGE_CROPS', 'customers_out', 'duration_hours',

    # NLP-derived features
    'desc_word_count', 'desc_char_count',
    'has_tornado', 'has_hail', 'has_flood', 'has_wind', 'has_tree',
    'has_power', 'has_damage', 'has_outage', 'has_broken', 'has_blown',

    #prism features
    'tmin', 'tmax', 'tavg', 'ppt'
]

# Features and target
X = storm_df_with_1_predictions[severity_features]
y = storm_df_with_1_predictions['severity_class']

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled = scaler.transform(X)

In [6]:
# Train Random Forest model on all data with best hyperparameters obtained using optuna
rf_model = RandomForestClassifier(
    n_estimators=201,
    max_depth=38,
    min_samples_split=4,
    min_samples_leaf=1,
    max_features='sqrt',
    bootstrap=False,
    random_state=42,
    n_jobs=-1
)
rf_model.fit(X_scaled, y)

# Predict on all data
predictions = rf_model.predict(X_scaled)

# Create new DataFrame with predictions
storm_data_with_severity = storm_df_with_1_predictions.copy()
storm_data_with_severity['severity_predicted'] = predictions

In [7]:
# 71% --> 86.5% (after tuning)****** TOO MUCH INC, SHOULD WE USE THIS INSTEAD ??

# from sklearn.preprocessing import StandardScaler, LabelEncoder
# from xgboost import XGBClassifier

# # Train XGBoost classifier on all data
# xgb_model = XGBClassifier(
#     n_estimators=403,
#     max_depth=10,
#     learning_rate=0.09065400280278058,
#     subsample=0.933968095670629,
#     colsample_bytree=0.5647574078202744,
#     gamma=0.00017586655077512627,
#     min_child_weight=2,
#     use_label_encoder=False,
#     eval_metric='logloss',
#     random_state=42
# )
# xgb_model.fit(X_scaled, y_encoded)

# # Predict on the full dataset
# predictions_encoded = xgb_model.predict(X_scaled)
# predictions = le.inverse_transform(predictions_encoded)

# storm_data_with_severity = storm_df_with_1_predictions.copy()
# storm_data_with_severity['severity_predicted'] = predictions


In [8]:
# Create three DataFrames based on predicted severity
df_low = storm_data_with_severity[storm_data_with_severity['severity_predicted'] == 0]
df_medium = storm_data_with_severity[storm_data_with_severity['severity_predicted'] == 1]
df_high = storm_data_with_severity[storm_data_with_severity['severity_predicted'] == 2]

# Outage Prediction

In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import lightgbm as lgb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Define outage features (common for all models)
outage_features = [
    'tmin', 'tmax', 'tavg', 'ppt',
    'has_tornado', 'has_hail', 'has_flood', 'has_wind', 'has_tree',
    'DAMAGE_PROPERTY', 'DAMAGE_CROPS', 'duration_hours',
    'desc_word_count', 'desc_char_count'
]

# Derive is_outage target (assuming customers_out > 0 indicates an outage)
for df in [df_low, df_medium, df_high]:
    df['is_outage'] = (df['customers_out'] > 0).astype(int)

2025-04-30 15:33:40.281935: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746027220.471137      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746027220.525573      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/tmp/ipykernel_31/1656639219.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['is_outage'] = (df['customers_out'] > 0).astype(int)
/tmp/ipykernel_31/1656639219.py:21: SettingWithCopyWarn

In [10]:
# Extract non-storm rows from storm_df_with_predictions
non_storm_data = storm_df_with_predictions[storm_df_with_predictions['predicted_storm'] == 0].copy()
non_storm_data['is_outage'] = 0  # No outage for non-storm events
non_storm_data['severity_predicted'] = 10  # Placeholder for non-storm rows

In [11]:
# Randomly split non-storm rows across the three DataFrames
non_storm_split = np.array_split(non_storm_data.sample(frac=1, random_state=42), 3)
non_storm_low, non_storm_medium, non_storm_high = non_storm_split

/usr/local/lib/python3.11/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [12]:
# Augment DataFrames with non-storm rows
df_low = pd.concat([df_low, non_storm_low], ignore_index=True)
df_medium = pd.concat([df_medium, non_storm_medium], ignore_index=True)
df_high = pd.concat([df_high, non_storm_high], ignore_index=True)

### Low Severity

In [13]:
# 1. LightGBM for Low Severity

if not df_low.empty:
    X_low = df_low[outage_features]
    y_low = df_low['is_outage']
    
    # Train-test split
    X_train_low, X_test_low, y_train_low, y_test_low = train_test_split(
        X_low, y_low, test_size=0.2, stratify=y_low, random_state=42
    )
    
    # Feature scaling
    scaler_low = StandardScaler()
    X_train_low_scaled = scaler_low.fit_transform(X_train_low)
    X_test_low_scaled = scaler_low.transform(X_test_low)

    # Best parameters from Optuna Trial 32
    best_params = {
        'n_estimators': 319,
        'learning_rate': 0.047847333909262976,
        'num_leaves': 281,
        'max_depth': 20,
        'min_child_samples': 22,
        'subsample': 0.527136639688917,
        'colsample_bytree': 0.8326768083509417,
        'random_state': 42
    }

    # Train final LightGBM model
    final_lgb_model = lgb.LGBMClassifier(**best_params)
    final_lgb_model.fit(X_train_low_scaled, y_train_low)
    
    # Predictions
    low_predictions = final_lgb_model.predict(X_test_low_scaled)
    
    # Evaluation
    print("Final Tuned LightGBM (Low Severity) Metrics:")
    print("Accuracy:", accuracy_score(y_test_low, low_predictions))
    print("Classification Report:\n", classification_report(y_test_low, low_predictions))
    print("Confusion Matrix:\n", confusion_matrix(y_test_low, low_predictions))


[LightGBM] [Info] Number of positive: 25768, number of negative: 27885
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004517 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2096
[LightGBM] [Info] Number of data points in the train set: 53653, number of used features: 14
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.480271 -> initscore=-0.078955
[LightGBM] [Info] Start training from score -0.078955
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Final Tuned LightGBM (Low Severity) Metrics:
Accuracy: 0.9513195169226182
Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.94      0.95      6972
           1       0.94      0.96      0.95      6442

    accuracy                  

In [14]:
from xgboost import XGBClassifier

# 2. XGBoost for Low Severity
if not df_low.empty:
    X_low = df_low[outage_features]
    y_low = df_low['is_outage']
    
    # Train-test split
    X_train_low, X_test_low, y_train_low, y_test_low = train_test_split(
        X_low, y_low, test_size=0.2, stratify=y_low, random_state=42
    )
    
    # Scale features
    scaler_low = StandardScaler()
    X_train_low_scaled = scaler_low.fit_transform(X_train_low)
    X_test_low_scaled = scaler_low.transform(X_test_low)
    
    # Train XGBoost with best hyperparameters
    xgb_model = XGBClassifier(
        n_estimators=250,
        max_depth=10,
        learning_rate=0.06249125027083313,
        subsample=0.5011230522934016,
        colsample_bytree=0.7916700511130836,
        gamma=1.0313638666099596e-05,
        min_child_weight=1,
        use_label_encoder=False,
        eval_metric='logloss',
        random_state=42
    )
    xgb_model.fit(X_train_low_scaled, y_train_low)
    
    # Predict on test set
    low_predictions = xgb_model.predict(X_test_low_scaled)
    
    # Evaluate
    print("XGBoost (Low Severity, Tuned) Metrics:")
    print("Accuracy:", accuracy_score(y_test_low, low_predictions))
    print("Classification Report:\n", classification_report(y_test_low, low_predictions))
    print("Confusion Matrix:\n", confusion_matrix(y_test_low, low_predictions))


XGBoost (Low Severity, Tuned) Metrics:
Accuracy: 0.94379007007604
Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.93      0.95      6972
           1       0.93      0.96      0.94      6442

    accuracy                           0.94     13414
   macro avg       0.94      0.94      0.94     13414
weighted avg       0.94      0.94      0.94     13414

Confusion Matrix:
 [[6498  474]
 [ 280 6162]]


In [15]:
# 3. Random Forest for Low Severity
if not df_low.empty:
    X_low = df_low[outage_features]
    y_low = df_low['is_outage']
    
    # Train-test split
    X_train_low, X_test_low, y_train_low, y_test_low = train_test_split(
        X_low, y_low, test_size=0.2, stratify=y_low, random_state=42
    )
    
    # Scale features
    scaler_low = StandardScaler()
    X_train_low_scaled = scaler_low.fit_transform(X_train_low)
    X_test_low_scaled = scaler_low.transform(X_test_low)
    
    # Train Random Forest
    rf_model = RandomForestClassifier(
        n_estimators=298,
        max_depth=38,
        min_samples_split=2,
        min_samples_leaf=1,
        max_features='sqrt',
        bootstrap=False,
        random_state=42,
        n_jobs=-1
    )
    rf_model.fit(X_train_low_scaled, y_train_low)
    
    # Predict on test set
    low_predictions = rf_model.predict(X_test_low_scaled)
    
    # Evaluate
    print("\nRandom Forest (Low Severity) Metrics:")
    print("Accuracy:", accuracy_score(y_test_low, low_predictions))
    print("Classification Report:\n", classification_report(y_test_low, low_predictions))
    print("Confusion Matrix:\n", confusion_matrix(y_test_low, low_predictions))


Random Forest (Low Severity) Metrics:
Accuracy: 0.9563142984941106
Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.96      0.96      6972
           1       0.95      0.95      0.95      6442

    accuracy                           0.96     13414
   macro avg       0.96      0.96      0.96     13414
weighted avg       0.96      0.96      0.96     13414

Confusion Matrix:
 [[6677  295]
 [ 291 6151]]


### Medium Severity

In [16]:
from xgboost import XGBClassifier

if not df_medium.empty:
    X_medium = df_medium[outage_features]
    y_medium = df_medium['is_outage']
    
    # Train-test split
    X_train_medium, X_test_medium, y_train_medium, y_test_medium = train_test_split(
        X_medium, y_medium, test_size=0.2, stratify=y_medium, random_state=42
    )
    
    # Scale features (XGBoost can handle unscaled features, but keeping it for consistency)
    scaler_medium = StandardScaler()
    X_train_medium_scaled = scaler_medium.fit_transform(X_train_medium)
    X_test_medium_scaled = scaler_medium.transform(X_test_medium)
    
    # Train XGBoost model
    xgb_model = XGBClassifier(
        n_estimators=334,
        max_depth=10,
        learning_rate=0.09402077273852373,
        subsample=0.872939114369583,
        colsample_bytree=0.7338166831128137,
        gamma=2.050143715974591e-05,
        min_child_weight=3,
        use_label_encoder=False,
        eval_metric='logloss',
        random_state=42
    )
    xgb_model.fit(X_train_medium_scaled, y_train_medium)
    
    # Predict on test set
    xgb_predictions = xgb_model.predict(X_test_medium_scaled)
    
    # Evaluate
    print("\nXGBoost (Medium Severity) Metrics:")
    print("Accuracy:", accuracy_score(y_test_medium, xgb_predictions))
    print("Classification Report:\n", classification_report(y_test_medium, xgb_predictions))
    print("Confusion Matrix:\n", confusion_matrix(y_test_medium, xgb_predictions))



XGBoost (Medium Severity) Metrics:
Accuracy: 0.9276118830122054
Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.91      0.93      6958
           1       0.90      0.95      0.92      6069

    accuracy                           0.93     13027
   macro avg       0.93      0.93      0.93     13027
weighted avg       0.93      0.93      0.93     13027

Confusion Matrix:
 [[6345  613]
 [ 330 5739]]


In [17]:
# 2. Random Forest for Medium Severity

if not df_medium.empty:
    X_medium = df_medium[outage_features]
    y_medium = df_medium['is_outage']
    
    # Train-test split
    X_train_medium, X_test_medium, y_train_medium, y_test_medium = train_test_split(
        X_medium, y_medium, test_size=0.2, stratify=y_medium, random_state=42
    )
    
    # Scale features
    scaler_medium = StandardScaler()
    X_train_medium_scaled = scaler_medium.fit_transform(X_train_medium)
    X_test_medium_scaled = scaler_medium.transform(X_test_medium)
    
    # Train Random Forest
    rf_model = RandomForestClassifier(
        n_estimators=191,
        max_depth=28,
        min_samples_split=6,
        min_samples_leaf=2,
        max_features='log2',
        bootstrap=False,
        random_state=42,
        n_jobs=-1
    )
    rf_model.fit(X_train_medium_scaled, y_train_medium)
    
    # Predict on test set
    medium_predictions = rf_model.predict(X_test_medium_scaled)
    
    # Evaluate
    print("\nRandom Forest (Medium Severity) Metrics:")
    print("Accuracy:", accuracy_score(y_test_medium, medium_predictions))
    print("Classification Report:\n", classification_report(y_test_medium, medium_predictions))
    print("Confusion Matrix:\n", confusion_matrix(y_test_medium, medium_predictions))


Random Forest (Medium Severity) Metrics:
Accuracy: 0.936132647578107
Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.93      0.94      6958
           1       0.92      0.94      0.93      6069

    accuracy                           0.94     13027
   macro avg       0.94      0.94      0.94     13027
weighted avg       0.94      0.94      0.94     13027

Confusion Matrix:
 [[6460  498]
 [ 334 5735]]


### High Severity

In [18]:
# FNN for High Severity

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

if not df_high.empty:
    X_high = df_high[outage_features]
    y_high = df_high['is_outage']

    # Train-test split
    X_train_high, X_test_high, y_train_high, y_test_high = train_test_split(
        X_high, y_high, test_size=0.2, stratify=y_high, random_state=42
    )

    # Scale features
    scaler_high = StandardScaler()
    X_train_high_scaled = scaler_high.fit_transform(X_train_high)
    X_test_high_scaled = scaler_high.transform(X_test_high)

    # Optimized hyperparameters from Optuna
    n_units = [80, 112, 96]
    dropout_rate = 0.1677425146431672
    learning_rate = 0.001715383587455835
    batch_size = 64

    # Build optimized model
    model = Sequential()
    model.add(Dense(n_units[0], activation='relu', input_shape=(len(outage_features),)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(n_units[1], activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(n_units[2], activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(
        X_train_high_scaled, y_train_high,
        epochs=20,
        batch_size=batch_size,
        validation_split=0.2,
        verbose=0
    )

    # Predict on test set
    high_predictions = (model.predict(X_test_high_scaled) > 0.5).astype(int).flatten()

    # Evaluation
    print("\nOptimized FNN (High Severity) Metrics:")
    print("Accuracy:", accuracy_score(y_test_high, high_predictions))
    print("Classification Report:\n", classification_report(y_test_high, high_predictions))
    print("Confusion Matrix:\n", confusion_matrix(y_test_high, high_predictions))


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1746027260.375010      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1746027260.375616      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5
I0000 00:00:1746027263.749902     157 service.cc:148] XLA service 0x781d9c005a30 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1746027263.750482     157 service.cc:156]   St

418/418 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step

Optimized FNN (High Severity) Metrics:
Accuracy: 0.8710087489718089
Classification Report:
               precision    recall  f1-score   support

           0       0.91      0.83      0.87      6958
           1       0.83      0.92      0.87      6415

    accuracy                           0.87     13373
   macro avg       0.87      0.87      0.87     13373
weighted avg       0.87      0.87      0.87     13373

Confusion Matrix:
 [[5776 1182]
 [ 543 5872]]


In [19]:
# 2. Random Forest for High Severity
if not df_high.empty:
    X_medium = df_high[outage_features]
    y_medium = df_high['is_outage']
    
    # Train-test split
    X_train_medium, X_test_medium, y_train_medium, y_test_medium = train_test_split(
        X_medium, y_medium, test_size=0.2, stratify=y_medium, random_state=42
    )
    
    # Scale features
    scaler_medium = StandardScaler()
    X_train_medium_scaled = scaler_medium.fit_transform(X_train_medium)
    X_test_medium_scaled = scaler_medium.transform(X_test_medium)

    # Train Random Forest
    rf_model = RandomForestClassifier(
        n_estimators=225,
        max_depth=47,
        min_samples_split=2,
        min_samples_leaf=1,
        max_features='sqrt',
        bootstrap=False,
        random_state=42,
        n_jobs=-1
    )
    rf_model.fit(X_train_medium_scaled, y_train_medium)
    
    # Predict on test set
    medium_predictions = rf_model.predict(X_test_medium_scaled)
    
    # Evaluate
    print("\nRandom Forest (High Severity) Metrics:")
    print("Accuracy:", accuracy_score(y_test_medium, medium_predictions))
    print("Classification Report:\n", classification_report(y_test_medium, medium_predictions))
    print("Confusion Matrix:\n", confusion_matrix(y_test_medium, medium_predictions))


Random Forest (High Severity) Metrics:
Accuracy: 0.9404770806849623
Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.93      0.94      6958
           1       0.93      0.95      0.94      6415

    accuracy                           0.94     13373
   macro avg       0.94      0.94      0.94     13373
weighted avg       0.94      0.94      0.94     13373

Confusion Matrix:
 [[6467  491]
 [ 305 6110]]


In [20]:
# 3. XGBoost for High Severity
if not df_high.empty:
    X_high = df_high[outage_features]
    y_high = df_high['is_outage']
    
    # Train-test split
    X_train_high, X_test_high, y_train_high, y_test_high = train_test_split(
        X_high, y_high, test_size=0.2, stratify=y_high, random_state=42
    )
    
    # Scale features
    scaler_high = StandardScaler()
    X_train_high_scaled = scaler_high.fit_transform(X_train_high)
    X_test_high_scaled = scaler_high.transform(X_test_high)

    xgb_model = XGBClassifier(
        n_estimators = 422, 
        max_depth = 10, 
        learning_rate = 0.06603920004888718, 
        subsample = 0.9760195263062684, 
        colsample_bytree = 0.6502775826898155, 
        gamma = 1.0450200674334978e-05,
        min_child_weight = 3,
        use_label_encoder=False,
        eval_metric='logloss',
        random_state=42
    )
    xgb_model.fit(X_train_high_scaled, y_train_high)
    
    # Predict on test set
    high_predictions = xgb_model.predict(X_test_high_scaled)
    
    # Evaluate
    print("\nRandom Forest (High Severity) Metrics:")
    print("Accuracy:", accuracy_score(y_test_high, high_predictions))
    print("Classification Report:\n", classification_report(y_test_high, high_predictions))
    print("Confusion Matrix:\n", confusion_matrix(y_test_high, high_predictions))


Random Forest (High Severity) Metrics:
Accuracy: 0.933597547296792
Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.91      0.93      6958
           1       0.91      0.96      0.93      6415

    accuracy                           0.93     13373
   macro avg       0.93      0.93      0.93     13373
weighted avg       0.93      0.93      0.93     13373

Confusion Matrix:
 [[6342  616]
 [ 272 6143]]


In [22]:
import optuna
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
import numpy as np

def objective(trial):
    # Hyperparameter search space
    n_units1 = trial.suggest_int("n_units1", 32, 128)
    n_units2 = trial.suggest_int("n_units2", 32, 128)
    n_units3 = trial.suggest_int("n_units3", 32, 128)
    dropout_rate = trial.suggest_float("dropout_rate", 0.1, 0.5)
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True)
    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128])

    # Data split and scaling
    X_high = df_high[outage_features]
    y_high = df_high['is_outage']
    X_train, X_val, y_train, y_val = train_test_split(
        X_high, y_high, test_size=0.2, stratify=y_high, random_state=42
    )
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)

    # Model definition
    model = Sequential()
    model.add(Dense(n_units1, activation='relu', input_shape=(X_train_scaled.shape[1],)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(n_units2, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(n_units3, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])

    # Train the model
    history = model.fit(
        X_train_scaled, y_train,
        validation_data=(X_val_scaled, y_val),
        epochs=20,
        batch_size=batch_size,
        verbose=0
    )

    # Predict and return validation F1 score
    preds = (model.predict(X_val_scaled) > 0.5).astype(int)
    return f1_score(y_val, preds)

# Only run tuning if data is available
if not df_high.empty:
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=30)

    print("Best trial:")
    print(study.best_trial)


[I 2025-04-30 15:35:27,868] A new study created in memory with name: no-name-faa9a608-00f7-454a-84a3-2eabc628766d
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


418/418 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


[I 2025-04-30 15:35:48,274] Trial 0 finished with value: 0.8596759400794864 and parameters: {'n_units1': 70, 'n_units2': 71, 'n_units3': 72, 'dropout_rate': 0.34659494980215777, 'learning_rate': 0.008275497099703811, 'batch_size': 128}. Best is trial 0 with value: 0.8596759400794864.
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


418/418 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


[I 2025-04-30 15:36:08,222] Trial 1 finished with value: 0.8652492777242758 and parameters: {'n_units1': 34, 'n_units2': 46, 'n_units3': 103, 'dropout_rate': 0.2831578771610723, 'learning_rate': 0.004415347921959894, 'batch_size': 128}. Best is trial 1 with value: 0.8652492777242758.
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

418/418 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


[I 2025-04-30 15:58:10,196] Trial 28 finished with value: 0.874450341167551 and parameters: {'n_units1': 127, 'n_units2': 113, 'n_units3': 118, 'dropout_rate': 0.13442637898797585, 'learning_rate': 0.0027934241225777943, 'batch_size': 128}. Best is trial 26 with value: 0.8753085496297404.
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


418/418 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


[I 2025-04-30 15:58:30,367] Trial 29 finished with value: 0.8733958724202627 and parameters: {'n_units1': 127, 'n_units2': 97, 'n_units3': 128, 'dropout_rate': 0.12977025968024897, 'learning_rate': 0.005777122511596615, 'batch_size': 128}. Best is trial 26 with value: 0.8753085496297404.


Best trial:
FrozenTrial(number=26, state=1, values=[0.8753085496297404], datetime_start=datetime.datetime(2025, 4, 30, 15, 55, 56, 120171), datetime_complete=datetime.datetime(2025, 4, 30, 15, 56, 53, 661818), params={'n_units1': 110, 'n_units2': 117, 'n_units3': 116, 'dropout_rate': 0.10607662795002712, 'learning_rate': 0.003063903238066634, 'batch_size': 32}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'n_units1': IntDistribution(high=128, log=False, low=32, step=1), 'n_units2': IntDistribution(high=128, log=False, low=32, step=1), 'n_units3': IntDistribution(high=128, log=False, low=32, step=1), 'dropout_rate': FloatDistribution(high=0.5, log=False, low=0.1, step=None), 'learning_rate': FloatDistribution(high=0.01, log=True, low=0.0001, step=None), 'batch_size': CategoricalDistribution(choices=(32, 64, 128))}, trial_id=26, value=None)
